# Retrieval API DEMO

## Code Initialisation
Dependencies and environment initialisation. Ensure there's a `.env` file with your credentials in the same directory as this script. Use the `env` file as reference.

In [2]:
import os
import json
import requests as r
from IPython.display import Markdown
import utils as u
from dotenv import load_dotenv

load_dotenv()

True

## Constants

In [3]:
API_HOST = 'api.dowjones.com'
AUTH_HOST = 'accounts.dowjones.com'
CLIENT_ID = os.getenv('FACTIVA_CLIENTID')
USERNAME = os.getenv('FACTIVA_USERNAME')
PASSWORD = os.getenv('FACTIVA_PASSWORD')
AUTH_URL = f"https://{AUTH_HOST}/oauth2/v1/token"
GCP_PROJECT = os.getenv('GCP_PROJECT')
GCP_LOCATION = os.getenv('GCP_LOCATION')

## Authentication - Generate Bearer

For details about getting the `bearer_token`, please see the `utils.py` file.

In [4]:
bearer_token = u.get_bearer_token(CLIENT_ID, USERNAME, PASSWORD, AUTH_URL)
if bearer_token:
    display(Markdown(f"**Authentication Successful**: Bearer token OK for user {USERNAME.split('@')[0]}"))
else:
    display(Markdown('**Authentication Failed**: Cannot obtain the Bearer token'))

**Authentication Successful**: Bearer token OK for user 9ZZZ159100-svcaccount

In [5]:
req_headers = {
    "Authorization": f"Bearer {bearer_token}",
    "Content-Type": "application/json"
}

## Factiva Retrieval API Query

### Prompt
Add the prompt to be sent to the Retrieval API

In [6]:
# "What are NASA's planned missions to the Moon in 2025, and what are their primary objectives?"
# "中国2025年计划进行哪些月球探测任务？其主要目标是什么？"   # What are China's planned missions to the Moon in 2025, and what are their primary objectives?
# "今後12か月間の日本の半導体業界の見通しはどのようなものですか?"  # What are the perspectives for the semiconductors industry in Japan for the next 12 months?
# "Summarise the latest earnings report from Microsoft Corp"
# "What are the perspectives of bitcoin for the next 12 months?"
frapi_prompt = "What are NASA's planned missions to the Moon in 2025, and what are their primary objectives?"

Assemble the Retrieval API payload

In [7]:
frapi_query = {
  "data": {
    "attributes": {
      "response_limit": 5,
      "query": {
        "search_filters": [
          {
            "scope": "Language",
            "value": "en"       # en or ja or de or it or es
          }
        ],

        "value": frapi_prompt
      }
    },
    "id": "GenAIRetrievalExample",
    "type": "genai-content"
  }
}


## Send Query and Receive Chunks from the Retrieval API

In [8]:
chunks_resp = r.post(f"https://{API_HOST}/content/gen-ai/retrieve", json=frapi_query, headers=req_headers)

if chunks_resp.status_code == 200:
    print('Successfully retrieved chunks')
else:
    print(f"Request Failed: {chunks_resp.json()}")

Successfully retrieved chunks


## Print Chunks

In [9]:
chunks_list = chunks_resp.json()['data']

# u.print_full_chunks(chunks_list)
u.print_partial_chunks(chunks_list)

###                            NASA revealed the missions it will carry out during 2025

**CE NoticiasFinancieras** - 2025-01-27 - NFINCE0020250127el1r004n7 - Lang: en

The directors of the North American Aerospace Agency (NASA) reported that in 2025 it will focus on consolidating its leadership in space exploration t...

---

### The most anticipated space missions of 2025

...

---

### From new commercial Moon landers to asteroid investigations, expect a slate of exciting space missions in 2025

**The Conversation** - 2024-12-26 - CONVAU0020241226ekcq00007 - Lang: en

Zhenbo Wang, University of Tennessee From new commercial Moon landers to asteroid investigations, expect a slate of exciting space missions in 2025 Zh...

---

## Retrieval API Conclusion

Up to this point it the seen functionalities are connected with the Retrieval API.

However, given that this service is an intermediate component in a full-stack solution, the below Test LLM steps and the [Read Article](2_read_article.ipynb) notebook are the two complementary avenues to get a full working solution.

## Test LLM

This is a downstream step that is only illustrative on how the response generation stage can be implemented. In the below example only a few articles are used to respond the prompt using an LLM hosted in **Google Cloud Vertex AI**. The passed request is built from the original prompt enhanced with the retrieved articles as gounding context.

The tested LLM is Google Gemini 2.0 Flash. Response generations took a few seconds for a context between 3K to 4K LLM tokens.

### Prerequisite

To successfully complete the following steps, it's important that there's an active GCloud authentication in the environment where this notbeook is executed from. This can be done by running the following command:

```bash
$ gcloud auth application-default login
```

For more information see [Set up ADC for a local development environment](https://cloud.google.com/docs/authentication/set-up-adc-local-dev-environment).

### Gemini Structured Prompt

In [10]:
instructions_text ="""
    You are an experienced business analyst that respond in a professional manner.
    Answer the query using only the information provided in the list of articles.
    If you use information from an article, cite it using squared brackets containing the index number.
    At the end of the answer, show a list of the cited articles ordered by index and under the title Cited Articles.
    Each cited article must be displayed in the following Markdown format:
    - [index] [headline - source_name - publication_date](url)
    Use Markdown for the output.
"""
article_list = []

for chunk in chunks_list:
    article = {
        'index': len(article_list) + 1,
        'url': f"https://dj.factiva.com/article?id=drn:archive.newsarticle.{str(chunk['meta']['original_doc_id']).strip()}",
        'source_name': str(chunk['meta']['source']['name']).strip(),
        'headline': str(chunk['attributes']['headline']['main']['text']).strip(),
        'publication_date': chunk['attributes']['publication_date'],
        'content': f"{str(chunk['attributes']['snippet']['content'][0]['text']).strip()} {str(chunk['attributes']['content'][0]['text']).strip()}"
    }
    article_list.append(article)

llm_prompt = {
    'query': frapi_prompt,
    'articles': article_list,
    "instructions": instructions_text.strip()
}

In [14]:
# u.print_full_llm_prompt(llm_prompt)
u.print_partial_llm_prompt(llm_prompt)

{
    "query": "What are NASA's planned missions to the Moon in 2025, and what are their primary objectives?",
    "articles": [
        {
            "index": 1,
            "url": "https://dj.factiva.com/article?id=drn:archive.newsarticle.NFINCE0020250127el1r004n7",
            "source_name": "CE NoticiasFinancieras",
            "headline": "NASA revealed the missions it will carry out during 2025",
            "publication_date": "2025-01-27",
            "content": "The directors of the North American Aerospace Agency (NASA) reported that in 2025 it will focus on consolidating its leadership in space exploration t..."
        },
        {
            "index": 2,
            "url": "https://dj.factiva.com/article?id=drn:archive.newsarticle.NFINCE0020250101ekcv000n2",
            "source_name": "CE NoticiasFinancieras",
            "headline": "The most anticipated space missions of 2025",
            "publication_date": "2024-12-31",
            "content": "In the coming months, di

### Gemini Request

In [13]:
response = u.gemini_generate(llm_prompt, GCP_PROJECT, GCP_LOCATION)
display(Markdown(f"**Prompt:** {frapi_prompt} \n --------------------"))
display(Markdown(response))

**Prompt:** What are NASA's planned missions to the Moon in 2025, and what are their primary objectives? 
 --------------------

In 2025, NASA has several planned missions to the Moon with the primary objective of laying the groundwork for a sustainable human presence and future exploration [1, 3]. These missions include:

*   **IM-2 Mission:** Scheduled by Intuitive Machines to land at the lunar south pole, this mission will drill into the lunar subsurface and measure volatiles using an advanced drill and mass spectrometer [1, 3]. This is a crucial test for future Artemis manned missions [1, 3]. The spacecraft will also carry a small satellite to map water deposits on the Moon to help NASA identify future landing sites for the Artemis mission [2].
*   **Commercial Lunar Payload Services (CLPS) program:** This program will continue to play a central role in developing technologies and scientific experiments to facilitate a sustainable human presence on the Moon [1, 3, 5]. Missions include deliveries by companies such as Astrobotic, Intuitive Machines, and Firefly Aerospace [5].
*   **Blue Ghost Mission:** Developed by Firefly Aerospace, this mission will spend two weeks on the Moon as part of NASA's Commercial Lunar Cargo Services [4]. It aims to carry a vehicle with 10 payloads from NASA and will head for Mons Latreille, gathering information about the lunar surface and how lunar rocks interact with the solar wind and Earth's magnetic field [2].
*   **M2/Resilience mission:** Japan’s mission will launch a lander and micro-rover to the lunar surface to study the lunar soil to understand its composition and properties. Researchers will also conduct a water-splitting test to produce oxygen and hydrogen by extracting water from the lunar surface [5].

These missions are critical to the success of the Artemis program and aim to expand human knowledge and consolidate strategic alliances with international and private partners [1, 3].

Cited Articles:

*   [1] [NASA revealed the missions it will carry out during 2025 - CE NoticiasFinancieras - 2025-01-27](https://dj.factiva.com/article?id=drn:archive.newsarticle.NFINCE0020250127el1r004n7)
*   [2] [The most anticipated space missions of 2025 - CE NoticiasFinancieras - 2024-12-31](https://dj.factiva.com/article?id=drn:archive.newsarticle.NFINCE0020250101ekcv000n2)
*   [3] [What are the most important missions to be carried out by NASA in 2025? - CE NoticiasFinancieras - 2025-01-04](https://dj.factiva.com/article?id=drn:archive.newsarticle.NFINCE0020250104el14009ja)
*   [4] [Find out what the space missions are for 2025 - CE NoticiasFinancieras - 2025-01-04](https://dj.factiva.com/article?id=drn:archive.newsarticle.NFINCE0020250104el1400abk)
*   [5] [From new commercial Moon landers to asteroid investigations, expect a slate of exciting space missions in 2025 - The Conversation - 2024-12-26](https://dj.factiva.com/article?id=drn:archive.newsarticle.CONVAU0020241226ekcq00007)
